In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm
import time
import scipy.io as io

import skimage.measure
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint
from keras.models import *
from keras.layers import *
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import model_from_json
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenetv2 import preprocess_input

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
np.random.seed(16)
tf.random.set_seed(16)
random.seed(16)

In [ ]:
# Images from Mall dataset

mall_gt = io.loadmat(".../mall_dataset/mall_gt.mat")
mall_head_pos = mall_gt['frame'][0]

path_frames_mall = '.../frames224/'
mall_names = sorted([file for file in os.listdir(path=path_frames_mall) if file.endswith('.jpg')])

Mall_imgs = []
Mall_labels = []

i=0

for img_name in sorted(os.listdir(path = path_frames_mall)):
    i=i+1
    if img_name in mall_names:
        img = preprocess_input(cv2.cvtColor(cv2.imread(path_frames_mall + img_name), cv2.COLOR_BGR2RGB))
        Mall_imgs.append(img)
        Mall_labels.append(len(mall_head_pos[i-1][0][0][0]))

Mall_imgs = np.asarray(Mall_imgs)
Mall_labels = np.asarray(Mall_labels)

In [ ]:
# Images from Shanghai dataset

Train_imgs_B = []
Train_labels_B = []

path_images = ".../shanghai224/images224/"
path_heads = ".../shanghai224/ground_truth/"


for img_name, heads_name in zip(sorted(os.listdir(path = path_images)), sorted(os.listdir(path = path_heads))):

    img = preprocess_input(cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB))
    Train_imgs_B.append(img)

    Train_labels_B.append(len(loadmat(path_heads + heads_name)['image_info'][0, 0][0, 0][0]))


Train_imgs_B = np.asarray(Train_imgs_B)
Train_labels_B = np.asarray(Train_labels_B)

In [ ]:
weights_dir = 'kaggle/working/weights'
os.makedirs(weights_dir, exist_ok=True)

# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, decay=5e-4), loss="mae", metrics=["mae"])

# Data Augmentation
#datagen = ImageDataGenerator(
#    rotation_range=20,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
#    fill_mode='nearest',
#    validation_split=0.2)  # 20% validation split

checkpoint_filepath = os.path.join(weights_dir, 'mobilenet_crowd_counting_best.weights.h5')
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mae',
    mode='min',
    save_best_only=True,
    verbose=1)

early_stopping = EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=5, min_lr=1e-6)

# Training on Shanghai dataset
history = model.fit(Train_imgs_B, Train_labels_B,
                    batch_size=16, epochs=200,
                    validation_split=0.1,
                    callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

final_weights_path = os.path.join(weights_dir, 'mobilenet_crowd_counting_final.weights.h5')
model.save('mobilenet_crowd_counting_final.weights.h5')

In [ ]:
Mall_imgs_train, Mall_imgs_test, Mall_labels_train, Mall_labels_test = train_test_split(
    Mall_imgs, Mall_labels, test_size=0.2, random_state=16
)

# Loading weights obtained by training the model on Shanghai dataset
model.load_weights(checkpoint_filepath)

# Training on Mall dataset
history_mall = model.fit(Mall_imgs_train, Mall_labels_train,
                         batch_size=16, epochs=100,
                         validation_split=0.1,
                         callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

final_weights_path_mall = os.path.join(weights_dir, 'mobilenet_crowd_counting_final_mall.weights.h5')
model.save_weights(final_weights_path_mall)

In [ ]:
def test_model(model, Test_imgs, Test_labels, file_parameters_path):
    model.load_weights(file_parameters_path)

    mae_error = 0
    mse_error = 0
    mape_error = 0


    N = len(Test_labels)
    for i, (x, y) in enumerate(zip(Test_imgs, Test_labels)):

        print(i)

        mae_error += np.abs(np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y)
        mse_error += np.square(np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y)
        mape_error += np.abs((np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y) / y)* 100

    return mae_error/N, np.sqrt(mse_error/N), mape_error/N

# Test on Mall dataset
mae, mse, mape  = test_model(model, Mall_imgs[:1200], Mall_labels[:1200], checkpoint_filepath)
print(f"Mean Absolute Error: {mae}, Mean Square Error: {mse}, Mean Absolute Percentage Error:{mape}")